In [1]:
%matplotlib inline

# Lecture 4: Tables, pt. II

In [2]:
import babypandas as bpd

# NBA Salaries

- The file `nba_salaries.csv` contains all salaries from 2015-2016 NBA season.

In [3]:
salaries = bpd.read_csv('nba_salaries.csv').set_index('PLAYER')
salaries

## Questions we can answer:

- What was the biggest salary?
- How many players were there?
- What was LeBron James' salary?

## Questions we'll learn how to answer today:

- *Who* had the biggest salary?
- Who was the highest paid center?
- What is the total payroll of the Cleveland Cavaliers?
- What team had the largest combined salary?

## Recall: getting a column

- use `.get(column_label)`
- Columns are not DataFrames, they're Series

In [4]:
salaries.get('2015_SALARY')

## Recall: getting a particular value from a column

- Use `.get(column_label).loc[row_label]`
- How much did LeBron James make?

In [5]:
salaries.get('2015_SALARY').loc['LeBron James']

## How to get a particular element from a table:

1. `.get()` the column label
2. `.loc[]` the row label

In this class, we'll always get column, then row (but row, then column is also possible).

 Example: What position does LeBron play?

In [6]:
#- use .get() then .loc[]


## Recall: Salary Analysis

- What was the biggest/smallest salary? What was the average salary?
- *Series* have helpful methods, like `.min()`, `.max()`, `.mean()`, etc.

In [7]:
salaries.get('2015_SALARY').min()

In [8]:
salaries.get('2015_SALARY').max()

In [9]:
salaries.get('2015_SALARY').mean()

## Use Case: *Who* had the biggest salary?

- Strategy: Sort the table by salary and take the name at the top

### Step 1) Sort the table

- Use the `.sort_values(by=column_name)` method to sort.
- **Notice:** Creates a new table.
- Everything works as expected, but we wanted *descending* order.

In [10]:
salaries.sort_values(by='2015_SALARY')

### Step 1) Sorting the table in *descending* order

- Use `.sort_values(by=column_name, ascending=False)` to sort in *descending* order

In [11]:
highest_salaries = salaries.sort_values(by='2015_SALARY', ascending=False)
highest_salaries

### Step 2) Get the *name* of the person with the highest salary

- We saw that is was Kobe, but how do we get the name using code?
- Remember, the index is an array

In [12]:
highest_salaries.index[0]

## Discussion Question

Suppose we didn't know how to sort in descending order. Which of these would still work?

- a) `salaries.sort_values(by='2015_SALARY')[max()]`
- b) `salaries.sort_values(by='2015_SALARY').index[0]`
- c) `salaries.sort_values(by='2015_SALARY').loc[max]`
- a) `salaries.sort_values(by='2015_SALARY').index[-1]`

## Answer: (d)

In [13]:
salaries.sort_values(by='2015_SALARY').index[-1]

## Use Case: What team did the person with the third-lowest salary play for?

- We have the tools, but its a little tricky. Can you think of a strategy?

## Strategy #1

1. Sort the table in ascending order using `.sort_values(by='ADJUSTED_SALARY')`
2. Get the name of the person using `.index[2]` (remember starts at 0)
3. Use `.get('TEAM').loc[their_name]` to get their team name.



In [14]:
lowest_salaries = salaries.sort_values(by='2015_SALARY')
lowest_salaries

In [15]:
name = lowest_salaries.index[2]
name

In [16]:
lowest_salaries.get('TEAM').loc[name]

## Another Approach

- Can we just get the 3rd element without knowing the label?
- Yes, with `.iloc[]`.
- `.iloc[]` takes integer position instead of row label.

In [17]:
lowest_salaries.get('TEAM')

In [18]:
lowest_salaries.get('TEAM').iloc[2]

## Strategy #2

1. Sort the table in ascending order using `.sort_values(by='ADJUSTED_SALARY')`, as before.
2. Use `.get('TEAM').iloc[2]` to get their team name.

In [19]:
salaries.sort_values(by='2015_SALARY').get('TEAM').iloc[2]

## Summary of accessing a Series

- There are two ways to get an element of a series:
    - `.loc[]` uses the row label
    - `.iloc[]` uses the integer position
- Usually `.loc` is more convenient

In [20]:
teams = lowest_salaries.get('TEAM')
teams

In [21]:
teams.loc['Thanasis Antetokounmpo']

In [22]:
teams.iloc[0]

## Note

- Sometimes the integer position and row label are the same
- This happens by default with `bpd.read_csv`:

In [23]:
bpd.read_csv('nba_salaries.csv')

In [24]:
bpd.read_csv('nba_salaries.csv').get('PLAYER').loc[3]

In [25]:
bpd.read_csv('nba_salaries.csv').get('PLAYER').iloc[3]

# Selecting Rows

## Use Case: Who was the highest paid center (C)?

In [26]:
salaries

## Selecting Rows

- We could do it if we had a table consisting only of centers.
- But how do we get that table?

## The Solution

In [27]:
salaries[salaries.get('POSITION') == 'C']

In [28]:
'PG' == 'C'

In [29]:
'C' == 'C'

In [30]:
salaries.get('POSITION') == 'C'

## Boolean Indexing

To select only some rows of `salaries`:

1. Make a list/array/Series of `True`s (keep) and `Falses` (toss)
2. Then pass it into `salaries[]`.

Rather than making the list by hand, we usually make comparisons.

## Elementwise comparisons work as expected

In [31]:
salaries.get('2015_SALARY') > 5

In [32]:
#- make a table with only players who made more than 5 million


## Another example

In [33]:
#- get only the Cleveland Cavaliers


## When the conditions are not satisfied

In [34]:
salaries[salaries.get('TEAM') == 'San Diego Surfers']

## Use Case: Who was the highest paid center?

1. Extract a table of centers
2. Sort by salary
3. Return first thing in index

In [35]:
#- extract a table of centers


In [36]:
#- sort and return first thing in index


## Discussion Question

What was the total payroll of the Cleveland Cavaliers?

- a) `salaries[salaries.get('TEAM') == 'Cleveland Cavaliers'].get('2015_SALARY').sum()`
- b) `salaries.get('2015_SALARY').sum()[salaries.get('TEAM') == 'Cleveland Cavaliers']`
- c) `salaries['Cleveland Cavaliers'].get('2015_SALARY').sum()`

## Answer: a)

In [37]:
cavs = salaries[salaries.get('TEAM') == 'Cleveland Cavaliers']
cavs

In [38]:
#- use series method .sum()


# GroupBy

## Use Case: What team had the highest payroll?

- We can find the payroll for one team.
- But how can we find the payroll for *all* teams at once?
- Can we somehow group the rows of the table by team name?

In [39]:
salaries

## `.groupby`


In [40]:
salaries.groupby('TEAM').sum()

## `.groupby`

1. Use `.groupby(column_name)` to gather rows which have the same value in the column
2. Apply an *aggregation* function *within* each group

The aggregation is applied to each column.

Some aggregation functions: `.count()`, `.sum()`, `.mean()`, `.median()`, `.max()`, `.min()`

In [41]:
salaries

## Interesting thing # 1

- The index has changed to team names.
- In general, the new row labels are the *group labels* (the thing that is used to determine if two rows were in same group).

In [42]:
salaries.groupby('TEAM').sum()

## Interesting thing #2

- The `POSITION` and `TEAM` columns have disappeared. Why?

In [43]:
salaries.groupby('TEAM').sum()

## Disappearing columns

- The aggregation function is applied to each column.
- If it doesn't make sense to apply it to a particular column, that column will disappear.
- Example: can't sum strings, like in `POSITION` column.
- Surprising example: *can* `max()` strings (alphabetically last)

In [44]:
salaries.groupby('TEAM').max()

## Use Case: What team has the highest payroll?

1. Group by team.
2. Within each group, compute total salary.
3. Sort by salary, descending order.
4. Take the name of the first team.

In [45]:
#- group, sum, sort, index


## Use Case: How many players play each position?

In [46]:
#- group, count


## Better column names?

To rename a column

1. Add a new column with `.assign`
2. Drop the old column(s) with `.drop(by=list_of_column_labels)`

In [47]:
counts = each_position.assign(COUNT=each_position.get('TEAM'))
counts

In [48]:
counts.drop(columns=['TEAM', '2015_SALARY'])

# IMDB

In [49]:
imdb = bpd.read_csv('imdb.csv').set_index('Title')
imdb

## What was the highest rated movie of the 1990's?

In [50]:
#- group or select?


## How many movies appear from each decade?

In [51]:
imdb.groupby('Decade').count()

In [52]:
imdb.groupby('Decade').count().plot(y='Year')